In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P25', 'P26', 'P27', 'P28', 'P29']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp6.csv', index = True, header=True)

2020-04-27 16:31:26 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P25.tsv' as 'real location' with error 0.01
2020-04-27 16:31:26 INFO     Detected encoding: ascii
2020-04-27 16:31:26 INFO     Found 3200 rows and 2 columns
2020-04-27 16:31:26 DEBUG    Checking column names
2020-04-27 16:31:26 DEBUG    Index name 'index'
2020-04-27 16:31:26 DEBUG    Column name 'P25'
2020-04-27 16:31:26 INFO     Checking data format
2020-04-27 16:31:26 INFO     Column 'P25'
2020-04-27 16:31:26 INFO     count    3200.000000
2020-04-27 16:31:26 INFO     mean        0.005350
2020-04-27 16:31:26 INFO     std         0.830552
2020-04-27 16:31:26 INFO     min        -6.721161
2020-04-27 16:31:26 INFO     50%         0.084718
2020-04-27 16:31:26 INFO     max        10.233867
2020-04-27 16:31:26 INFO     ---
2020-04-27 16:31:26 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with err

Time: 408 sec.

2020-04-27 16:38:15 INFO     Extracting autoclass results
2020-04-27 16:38:15 INFO     Found 3200 cases classified in 19 classes
2020-04-27 16:38:19 INFO     Aggregating input data
2020-04-27 16:38:19 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:38:20 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P26.tsv' as 'real location' with error 0.01
2020-04-27 16:38:20 INFO     Detected encoding: ascii
2020-04-27 16:38:20 INFO     Found 3200 rows and 2 columns
2020-04-27 16:38:20 DEBUG    Checking column names
2020-04-27 16:38:20 DEBUG    Index name 'index'
2020-04-27 16:38:20 DEBUG    Column name 'P26'
2020-04-27 16:38:20 INFO     Checking data format
2020-04-27 16:38:20 INFO     Column 'P26'
2020-04-27 16:38:20 INFO     count    3200.000000
2020-04-27 16:38:20 INFO     mean        0.042552
2020-04-27 16:38:20 INFO     std         0.727256
2020-04-27 16:38:20 INFO     min        -7.862112
2020-04-27 16:38:20 INFO     50%   

[0.05980861244019139, 0.07857142857142857, 0.08771929824561403, 0.17754569190600522, 0.23125, 0.0815450643776824, 0.12857142857142856, 0.12269938650306748, 0.20512820512820512, 0.16793893129770993, 1.0, 0.15384615384615385, 0.16666666666666666, 0.21951219512195122, 0.796875, 0.14285714285714285, 0.18181818181818182, 0.23076923076923078, 0.2857142857142857]
['GOOGLE', 'CLOUDFLARE', 'HTTP_PROXY', 'CONTENT_FLASH', 'DROPBOX', 'MS_ONE_DRIVE', 'MS_ONE_DRIVE', 'HTTP', 'OFFICE_365', 'AMAZON', 'IP_ICMP', 'SSL', 'HTTP_CONNECT', 'APPLE', 'DNS', 'EBAY', 'MSN', 'DNS', 'DROPBOX']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
0.23783352125447083


2020-04-27 16:38:20 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:38:20 INFO     Writing run file
2020-04-27 16:38:20 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:38:20 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:38:20 INFO     Running clustering...


...

[0.07494646680942184, 0.11311053984575835, 0.06971677559912855, 0.08549222797927461, 0.09113924050632911, 0.11016949152542373, 0.10047846889952153, 0.07547169811320754, 0.2919254658385093, 1.0, 0.15789473684210525, 0.9791666666666666, 0.2413793103448276]
['APPLE_ICLOUD', 'DROPBOX', 'CLOUDFLARE', 'OFFICE_365', 'AMAZON', 'WHATSAPP', 'WINDOWS_UPDATE', 'HTTP_CONNECT', 'CONTENT_FLASH', 'IP_ICMP', 'HTTP', 'DNS', 'OFFICE_365']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
0.26083777607462877


2020-04-27 16:55:20 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:20 INFO     Writing run file
2020-04-27 16:55:20 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:20 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:55:20 INFO     Running clustering...


Time: 448 sec.

2020-04-27 17:02:48 INFO     Extracting autoclass results
2020-04-27 17:02:48 INFO     Found 3200 cases classified in 26 classes
2020-04-27 17:02:54 INFO     Aggregating input data
2020-04-27 17:02:54 INFO     Writing classes + probabilities .tsv file


[0.06605922551252848, 0.20496894409937888, 0.09570957095709572, 0.09584664536741214, 0.11320754716981132, 0.21904761904761905, 0.08441558441558442, 0.11428571428571428, 0.10317460317460317, 0.09090909090909091, 1.0, 0.14814814814814814, 0.18867924528301888, 0.2872340425531915, 0.13978494623655913, 0.2077922077922078, 0.1917808219178082, 0.14545454545454545, 0.1282051282051282, 0.11688311688311688, 0.19047619047619047, 1.0, 0.17647058823529413, 0.125, 0.1891891891891892, 0.375]
['DROPBOX', 'CONTENT_FLASH', 'APPLE_ICLOUD', 'APPLE_ICLOUD', 'APPLE', 'CLOUDFLARE', 'WINDOWS_UPDATE', 'HTTP_CONNECT', 'EBAY', 'CLOUDFLARE', 'IP_ICMP', 'DROPBOX', 'DNS', 'OFFICE_365', 'APPLE_ITUNES', 'SPOTIFY', 'HTTP', 'DROPBOX', 'GMAIL', 'WINDOWS_UPDATE', 'WHATSAPP', 'DNS', 'HTTP', 'HTTP_CONNECT', 'INSTAGRAM', 'HTTP']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
0.22298933520435524
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (